In [56]:
import pandas as pd

companies = pd.read_csv("companies.txt", delimiter="\t", encoding='ISO-8859-1')
round2 = pd.read_csv("rounds2.csv", encoding='ISO-8859-1')

# Setting the value of key column to lower case to ensure consistency with joining dataframe
companies['permalink'] = companies['permalink'].str.lower()
round2['company_permalink'] = round2['company_permalink'].str.lower()

# Replacing all NaN (Null) values with 0 to ensure that we get consistent aggregations
round2['raised_amount_usd'].fillna(0, inplace=True)

# Table 1.1 outcome
print("\n\nTable 1.1")
print("----------\n")

# Get details of unique companies in companies file
print("Unique values in companies DataFrame :", companies['permalink'].nunique())

# Get details of unique companies in round2
print("Unique values in companies round2 :", round2['company_permalink'].nunique())

# Find out of there are companies in round2 that are not present in companies file

df1 = pd.DataFrame(companies,columns=['permalink'])
df2 = pd.DataFrame(round2,columns=['company_permalink'])

set_diff_df = pd.concat([df2, df1, df1], sort=True).drop_duplicates(keep=False)
print("Additional companies in round2 that does not exist in companies DataFrame :", set_diff_df['company_permalink'].count())

# Merger two data frames
master_frame = pd.merge(companies, round2, how='outer', left_on=['permalink'], right_on=['company_permalink'])

# Table 2.1 outcome
print("\n\nTable 2.1")
print("----------\n")

# Setting display format for float values that are an outcome of groupby
pd.set_option('display.float_format', lambda x: '%.0f' %x)

# Calculate the most representative value of investment amount by funding type = 'VENTURE'
venture_usd = round2[round2['funding_round_type'] == 'venture'].groupby('funding_round_type')['raised_amount_usd'].mean()
print ("Investment amount by funding type as 'VENTURE'        = ", '%.0f' % venture_usd)

# Calculate the most representative value of investment amount by funding type = 'ANGEL'
angel_usd = round2[round2['funding_round_type'] == 'angel'].groupby('funding_round_type')['raised_amount_usd'].mean()
print ("Investment amount by funding type as 'ANGEL'          = ", '%.0f' % angel_usd)

# Calculate the most representative value of investment amount by funding type = 'SEED'
seed_usd = round2[round2['funding_round_type'] == 'seed'].groupby('funding_round_type')['raised_amount_usd'].mean()
print ("Investment amount by funding type as 'SEED'           = ", '%.0f' % seed_usd)

# Calculate the most representative value of investment amount by funding type = 'PRIVATE EQUITY'
private_equity_usd = round2[round2['funding_round_type'] == 'private_equity'].groupby('funding_round_type')['raised_amount_usd'].mean()
print ("Investment amount by funding type as 'PRIVATE EQUITY' = ", '%.0f' % private_equity_usd)


# Table 3.1 outcome
print("\n\nTable 3.1")
print("----------\n")

#For the chosen investment type, make a data frame named top9 with the top nine countries 
top9 = master_frame[master_frame['funding_round_type'] == 'venture'].groupby('country_code').sum()
print(top9['raised_amount_usd'].nlargest(9))

# Table 4.1 outcome
print("\n\nTable 4.1")
print("----------\n")

# Splitting category list to extract primary sector into a new column 
new_column = master_frame['category_list'].str.split('|', n=1, expand=True)
master_frame['primary_sector'] = new_column[0]

# Load mapping file
mappings = pd.read_csv("mapping.csv", delimiter=",", encoding='ISO-8859-1')

master_frame['primary_sector'].fillna(0, inplace=True)
mappings['category_list'].fillna(0, inplace=True)

# Map each primary sector mapped to its main sector 
master_frame_mapping = pd.merge(master_frame, mappings, how='inner', left_on=['primary_sector'], right_on=['category_list'])

print("\n\nTable 5.1")
print("----------\n")

dfD1 = master_frame_mapping[(master_frame_mapping['funding_round_type'] == 'venture') & (master_frame_mapping['country_code'] == 'USA')]
dfD1 = (dfD1.groupby('primary_sector')
        .agg({'raised_amount_usd':'sum',
              'Automotive & Sports':'count',
              'Blanks':'count',
              'Cleantech / Semiconductors':'count',
              'Entertainment':'count',
              'Health':'count',
              'Manufacturing':'count',
              'News, Search and Messaging':'count', 
              'Others':'count',
              'Social, Finance, Analytics, Advertising':'count'})
        .reset_index()
        .rename(columns={'Automotive & Sports':'Automotive & Sports Count',
                         'Blanks':'Blanks Count',
                         'Cleantech / Semiconductors':'Cleantech / Semiconductors Count',
                         'Entertainment':'Entertainment Count',
                         'Health':'Health count',
                         'Manufacturing':' Manufacturing count',
                         'News, Search and Messaging':'News, Search and Messaging Count', 
                         'Others':'Others Count',
                         'Social, Finance, Analytics, Advertising':'Social, Finance, Analytics, Advertising Count'}))

#print("C1: Total number of Investments (count)", dfD1.count())
#print("C1: Total amount of Investments (count)", dfD1.sum())

#dfD1.sort_values('raised_amount_usd', ascending=False, inplace=True)
print(dfD1.head(3))



Table 1.1
----------

Unique values in companies DataFrame : 66368
Unique values in companies round2 : 66370
Additional companies in round2 that does not exist in companies DataFrame : 42493


Table 2.1
----------

Investment amount by funding type as 'VENTURE'        =  10634054
Investment amount by funding type as 'ANGEL'          =  764564
Investment amount by funding type as 'SEED'           =  556607
Investment amount by funding type as 'PRIVATE EQUITY' =  62111788


Table 3.1
----------

country_code
USA   422510842796
CHN    39835418773
GBR    20245627416
IND    14391858718
CAN     9583332317
FRA     7259536732
ISR     6907514579
DEU     6346959822
JPN     3363676611
Name: raised_amount_usd, dtype: float64


Table 4.1
----------



Table 5.1
----------

  primary_sector  raised_amount_usd  Automotive & Sports Count  Blanks Count  \
0              0         2442813454                        338           338   
1             3D          385954865                         49     

In [1]:
%config IPCompleter.greedy = True